## 참고 사이트
- https://velog.io/@ezoo0422/Python-pycaret%EC%9D%84-%EC%82%AC%EC%9A%A9%ED%95%98%EC%97%AC-%EB%AA%A8%EB%8D%B8-%EC%84%A0%EC%A0%95%ED%95%98%EA%B8%B0
- https://ideal-dominate.medium.com/pycaret-regression-%EA%B0%99%EC%9D%B4-%ED%95%B4%EB%B4%90%EC%9A%94-eaf0e36268
- https://ideal-dominate.medium.com/%ED%8C%8C%EB%9E%80%EB%A7%8C%EC%9E%A5-pycaret-%EC%84%A4%EC%B9%98%EA%B8%B0-cf703f692e3a
- https://koreapy.tistory.com/744

- https://pycaret.gitbook.io/docs/get-started/quickstart

## pycaret 설치
- conda install -c conda-forge pycaret

In [ ]:
#pip install pycaret[full]

## data load

In [1]:
# from pycaret.datasets import get_data
# dataset = get_data('insurance')

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


random state를 789으로 해줘서 90% :10%으로 훈련 데이터, 테스트 데이터를 나눈다

In [7]:
import pandas as pd

In [69]:
dataset2 = pd.read_csv('./hospital_male_0725.csv', encoding='cp949')
dataset2.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
dataset2 = dataset2[['area','yyyy','mm','dd','frequency','elderly_pop','total_pop','elderly_ratio','avg_age','pm10','avg_rhm','rhm_min_avg_dif','avg_ta','min_max_Ta','avg_ps','min_max_ps']]
dataset2

,area,yyyy,mm,dd,frequency,elderly_pop,total_pop,elderly_ratio,avg_age,pm10,avg_rhm,rhm_min_avg_dif,avg_ta,min_max_Ta,avg_ps,min_max_ps
0,강원,2012.0,1.0,1.0,3,96281.0,772718.0,12.460044,39.4,86.279167,71.285714,23.088571,-4.226027,9.280556,1023.886667,6.340000
1,경기,2012.0,1.0,1.0,4,447450.0,6020446.0,7.432174,36.1,91.567357,73.827586,25.065517,-3.962264,9.245283,1024.875000,5.500000
2,경남,2012.0,1.0,1.0,2,152866.0,1665308.0,9.179443,37.4,59.554398,63.333333,19.133333,0.335897,10.074359,1022.950000,5.200000
3,경북,2012.0,1.0,1.0,6,167115.0,1354649.0,12.336406,39.5,76.618056,65.052632,17.005263,-0.666667,9.628070,1023.268000,5.688000
4,광주,2012.0,1.0,1.0,0,55792.0,726529.0,7.679253,35.3,44.745833,86.333333,15.400000,-0.200000,6.500000,1025.250000,4.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24832,전남,2015.0,12.0,31.0,0,154519.0,953881.0,16.198981,42.1,55.015673,80.300000,24.265854,4.156757,7.135135,1031.053846,6.892308
24833,전북,2015.0,12.0,31.0,1,135253.0,930255.0,14.539347,40.9,69.055014,86.266667,25.910417,1.583333,7.536667,1031.300000,7.014286
24834,제주,2015.0,12.0,31.0,0,34676.0,313428.0,11.063466,38.7,71.968750,73.903226,21.264516,5.128571,4.877143,1030.792308,7.523077
24835,충남,2015.0,12.0,31.0,0,142107.0,1054439.0,13.477024,40.2,51.015625,79.647059,24.624837,1.912121,7.154545,1031.513333,6.800000


In [70]:
data = dataset2.sample(frac=0.9, random_state=786) 
# data.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
data_unseen = dataset2.drop(data.index) 
data.reset_index(drop=True, inplace=True) 
data_unseen.reset_index(drop=True, inplace=True) 
print('Data for Modeling: ' + str(data.shape)) 
print('Unseen Data For Predictions: ' + str(data_unseen.shape)) 
# Data for Modeling: (5400, 8) # Unseen Data For Predictions: (600, 8)

Data for Modeling: (22353, 16)
Unseen Data For Predictions: (2484, 16)


## 모델 환경 설정 (Settinf up Environment in PyCaret)

- pycaret을 사용하기 전에 pycaret에 맞게 데이터를 설정
- set_up() 함수를 사용
- 기본적으로 data와 target을 입력

In [97]:
# 자동으로 데이터 유형 지정 
from pycaret.regression import *
s = setup(data, target = 'frequency')

,Description,Value
0,Session id,5520
1,Target,frequency
2,Target type,Regression
3,Data shape,"(22353, 16)"
4,Train data shape,"(15647, 16)"
5,Test data shape,"(6706, 16)"
6,Numeric features,14
7,Categorical features,1
8,Preprocess,True
9,Imputation type,simple


#### set_up(): pycaret을 사용하기 위한 data setting

- session_id: random_state와 같은 개념으로 같은 결과가 나올 수 있게 seed를 고정
- data: train 데이터 입력
- target = target 변수 이름을 입력

## 모델 생성 - 비교

- 여러 모델을 적합하여 성능을 비교하는 단계
- compare_models(): 다양한 모델 적합 후 성능 비교
    - fold: cross_validation의 fold를 지정 (default = 10)
    - sort: 정렬기준 지표 설정
    - n_select: 상위 n개의 모델 결과만 출력

In [72]:
best_model_5 = compare_models(sort='RMSE', n_select = 5)
best_model_5

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.9060,1.5299,1.2365,0.4575,0.4901,0.4391,0.2710
lr,Linear Regression,0.9075,1.5447,1.2425,0.4522,0.4898,0.4432,0.1830
ridge,Ridge Regression,0.9076,1.5447,1.2425,0.4522,0.4898,0.4432,0.0470
lar,Least Angle Regression,0.9074,1.5447,1.2425,0.4522,0.4898,0.4431,0.0470
br,Bayesian Ridge,0.9084,1.5449,1.2425,0.4522,0.4898,0.4432,0.0580
gbr,Gradient Boosting Regressor,0.8994,1.5454,1.2428,0.4516,0.4929,0.4608,3.0030
omp,Orthogonal Matching Pursuit,0.9060,1.5491,1.2442,0.4507,0.4899,0.4412,0.0460
en,Elastic Net,0.9380,1.5960,1.2630,0.4340,0.4965,0.4329,0.3100
lasso,Lasso Regression,0.9385,1.5975,1.2635,0.4335,0.4967,0.4327,0.2440
lightgbm,Light Gradient Boosting Machine,0.8747,1.6002,1.2646,0.4321,0.5038,0.4354,1.0530


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

[ExtraTreesRegressor(n_jobs=-1, random_state=519),
 LinearRegression(n_jobs=-1),
 Ridge(random_state=519),
 Lars(random_state=519),
 BayesianRidge()]

In [73]:
print(best_model_5)

[ExtraTreesRegressor(n_jobs=-1, random_state=519), LinearRegression(n_jobs=-1), Ridge(random_state=519), Lars(random_state=519), BayesianRidge()]


In [74]:
models()

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


## 모델 직접 생성

- 하나의 모델의 적합 결과를 보는 방법
- create_model(): 하나의 모델 적합

    - fold: cross_validation의 fold 지정 (default = 10)

In [23]:
rf = create_model('rf')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0142,0.0080,0.0897,0.9974,0.0144,0.0044
1,0.0152,0.0113,0.1064,0.9962,0.0183,0.0049
2,0.0123,0.0072,0.0848,0.9977,0.0123,0.0039
3,0.0143,0.0085,0.0924,0.9971,0.0153,0.0050
4,0.0132,0.0049,0.0702,0.9982,0.0148,0.0056
5,0.0137,0.0070,0.0834,0.9976,0.0110,0.0039
6,0.0104,0.0059,0.0770,0.9979,0.0142,0.0040
7,0.0128,0.0062,0.0786,0.9976,0.0141,0.0049
8,0.0176,0.0091,0.0953,0.9970,0.0195,0.0058


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

## 모델 튜닝 - tuning
- tune_model() 함수 사용
- tune_model(  estimator = None, 
               fold = 10, 
               round = 4, 
               n_iter = 10,
               custom_grid = None, # 커스텀 파라미터 dict
               optimize = 'Accuracy', # 튜닝을 통해 최적화하고자 하는 대상 기본값은 정확도
               custom_scorer = None, # sklearn.make_scorer 로 제작된 메소드만 가능
               choose_better = False,
               verbose = True)

In [96]:
#tuned_rf = tune_model(rf)

## 모델 튜닝 - blending
- 생성한 모델끼리의 혼합

In [75]:
# 방법 1
# dt = create_model('dt')
# rf = create_model('rf')

# blender_2 = blend_models(estimator_list = [dt, rf])

# 방법 2
best_model_5 = compare_models(sort='RMSE', n_select=5)
blender_5 = blend_models(best_model_5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.9060,1.5299,1.2365,0.4575,0.4901,0.4391,0.2580
lr,Linear Regression,0.9075,1.5447,1.2425,0.4522,0.4898,0.4432,0.0570
ridge,Ridge Regression,0.9076,1.5447,1.2425,0.4522,0.4898,0.4432,0.0450
lar,Least Angle Regression,0.9074,1.5447,1.2425,0.4522,0.4898,0.4431,0.0480
br,Bayesian Ridge,0.9084,1.5449,1.2425,0.4522,0.4898,0.4432,0.0540
gbr,Gradient Boosting Regressor,0.8994,1.5454,1.2428,0.4516,0.4929,0.4608,2.9990
omp,Orthogonal Matching Pursuit,0.9060,1.5491,1.2442,0.4507,0.4899,0.4412,0.0450
en,Elastic Net,0.9380,1.5960,1.2630,0.4340,0.4965,0.4329,0.2990
lasso,Lasso Regression,0.9385,1.5975,1.2635,0.4335,0.4967,0.4327,0.2250
lightgbm,Light Gradient Boosting Machine,0.8747,1.6002,1.2646,0.4321,0.5038,0.4354,0.9300


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9010,1.5179,1.2320,0.4231,0.4964,0.4507
1,0.8870,1.5336,1.2384,0.4536,0.4862,0.4293
2,0.9191,1.5812,1.2575,0.4842,0.4969,0.4409
3,0.9370,1.6571,1.2873,0.4431,0.4973,0.4488
4,0.8974,1.5106,1.2291,0.4080,0.4806,0.4494
5,0.9009,1.5107,1.2291,0.4994,0.4797,0.4248
6,0.8778,1.4108,1.1878,0.4681,0.4883,0.4199
7,0.8996,1.4533,1.2055,0.4567,0.4842,0.4366
8,0.9073,1.5481,1.2442,0.4497,0.4881,0.4658


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [76]:
final_model = finalize_model(blender_5)
prediction = predict_model(final_model, data = data_unseen)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0.7502,1.0562,1.0277,0.6306,0.4175,0.3618


In [77]:
prediction

,area,yyyy,mm,dd,elderly_pop,total_pop,elderly_ratio,avg_age,pm10,avg_rhm,rhm_min_avg_dif,avg_ta,min_max_Ta,avg_ps,min_max_ps,frequency,Label
0,4.614449,2012.0,1.0,1.0,447450.0,6020446.0,7.432174,36.099998,91.567360,73.827583,25.065517,-3.962264,9.245283,1024.875000,5.500000,4,4.585968
1,1.146008,2012.0,1.0,1.0,105991.0,1251881.0,8.466539,37.200001,62.871376,54.000000,6.000000,0.200000,9.333333,1022.900024,5.150000,1,1.298594
2,1.428571,2012.0,1.0,1.0,136959.0,956024.0,14.325895,40.299999,60.954166,73.562500,16.306250,2.398630,6.673973,1024.681519,3.721429,3,1.826252
3,1.146008,2012.0,1.0,3.0,105991.0,1251881.0,8.466539,37.200001,59.842754,56.000000,21.299999,-0.566667,6.766667,1020.750000,5.700000,1,1.190972
4,1.104387,2012.0,1.0,4.0,96281.0,772718.0,12.460044,39.400002,73.483330,62.588234,16.859663,-8.506945,8.512676,1024.880005,8.460000,1,1.191953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,1.073473,2015.0,12.0,30.0,132897.0,1469869.0,9.041418,38.500000,61.215202,75.727272,18.145454,2.652381,5.952381,1028.275024,5.800000,1,1.058382
2480,1.161094,2015.0,12.0,30.0,135253.0,930255.0,14.539347,40.900002,58.884258,79.500000,31.350000,1.112903,12.416129,1029.421387,5.985714,0,0.915959
2481,4.613688,2015.0,12.0,31.0,562360.0,6299812.0,8.926616,37.900002,71.144722,83.470589,22.464706,0.409259,8.059259,1031.699951,7.150000,5,4.666992
2482,1.430851,2015.0,12.0,31.0,154519.0,953881.0,16.198980,42.099998,55.015675,80.300003,24.265854,4.156757,7.135135,1031.053833,6.892308,0,1.105181


In [79]:
dataset2

,area,yyyy,mm,dd,frequency,elderly_pop,total_pop,elderly_ratio,avg_age,pm10,avg_rhm,rhm_min_avg_dif,avg_ta,min_max_Ta,avg_ps,min_max_ps
0,강원,2012.0,1.0,1.0,3,96281.0,772718.0,12.460044,39.4,86.279167,71.285714,23.088571,-4.226027,9.280556,1023.886667,6.340000
1,경기,2012.0,1.0,1.0,4,447450.0,6020446.0,7.432174,36.1,91.567357,73.827586,25.065517,-3.962264,9.245283,1024.875000,5.500000
2,경남,2012.0,1.0,1.0,2,152866.0,1665308.0,9.179443,37.4,59.554398,63.333333,19.133333,0.335897,10.074359,1022.950000,5.200000
3,경북,2012.0,1.0,1.0,6,167115.0,1354649.0,12.336406,39.5,76.618056,65.052632,17.005263,-0.666667,9.628070,1023.268000,5.688000
4,광주,2012.0,1.0,1.0,0,55792.0,726529.0,7.679253,35.3,44.745833,86.333333,15.400000,-0.200000,6.500000,1025.250000,4.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24832,전남,2015.0,12.0,31.0,0,154519.0,953881.0,16.198981,42.1,55.015673,80.300000,24.265854,4.156757,7.135135,1031.053846,6.892308
24833,전북,2015.0,12.0,31.0,1,135253.0,930255.0,14.539347,40.9,69.055014,86.266667,25.910417,1.583333,7.536667,1031.300000,7.014286
24834,제주,2015.0,12.0,31.0,0,34676.0,313428.0,11.063466,38.7,71.968750,73.903226,21.264516,5.128571,4.877143,1030.792308,7.523077
24835,충남,2015.0,12.0,31.0,0,142107.0,1054439.0,13.477024,40.2,51.015625,79.647059,24.624837,1.912121,7.154545,1031.513333,6.800000


In [84]:
dataset3 = pd.read_csv('./hospital_male_0726_test.csv', encoding='cp949')
dataset3.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
dataset3 = dataset3[['area','yyyy','mm','dd','frequency','elderly_pop','total_pop','elderly_ratio','avg_age','pm10','avg_rhm','rhm_min_avg_dif','avg_ta','min_max_Ta','avg_ps','min_max_ps']]
dataset3

,area,yyyy,mm,dd,frequency,elderly_pop,total_pop,elderly_ratio,avg_age,pm10,avg_rhm,rhm_min_avg_dif,avg_ta,min_max_Ta,avg_ps,min_max_ps
0,강원,2011.0,12.0,1.0,3.0,95929.0,773278.0,12.405500,39.3,15.678261,83.354839,16.669545,1.281944,3.845070,1027.575000,2.993750
1,경기,2011.0,12.0,1.0,4.0,444422.0,6015357.0,7.388123,36.0,13.474891,65.448276,14.113793,5.360377,5.413208,1025.875000,3.175000
2,경남,2011.0,12.0,1.0,2.0,152283.0,1664755.0,9.147472,37.3,16.193461,74.583333,9.783333,6.394872,3.112821,1023.921429,2.578571
3,경북,2011.0,12.0,1.0,6.0,166547.0,1355727.0,12.284700,39.5,12.743785,81.000000,14.400000,4.429091,3.338182,1026.137500,2.684000
4,광주,2011.0,12.0,1.0,0.0,55360.0,725606.0,7.629485,35.3,8.566667,89.000000,12.066667,6.240000,3.840000,1022.900000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31581,전남,2016.0,12.0,31.0,NaN,158747.0,951889.0,16.677050,42.6,34.282500,79.909091,21.753535,2.539189,10.937838,1030.607692,2.573333
31582,전북,2016.0,12.0,31.0,NaN,139233.0,927505.0,15.011563,41.4,44.546347,87.722222,15.777778,1.012903,9.300000,1030.586667,3.466667
31583,제주,2016.0,12.0,31.0,NaN,36378.0,322545.0,11.278426,39.2,24.247294,67.741935,22.135484,3.997143,7.931429,1029.900000,3.046667
31584,충남,2016.0,12.0,31.0,NaN,146606.0,1064765.0,13.768860,40.7,43.536458,82.550000,17.345000,2.663636,7.596970,1030.100000,2.520000


In [85]:
dataset4 = dataset3[dataset3['yyyy'].astype(str).str.contains('2016.0')].reset_index(drop=True)
dataset4

,area,yyyy,mm,dd,frequency,elderly_pop,total_pop,elderly_ratio,avg_age,pm10,avg_rhm,rhm_min_avg_dif,avg_ta,min_max_Ta,avg_ps,min_max_ps
0,강원,2016.0,1.0,1.0,NaN,109543.0,781185.0,14.022671,41.4,54.629545,73.529412,19.665950,-1.454167,10.104167,1030.230000,8.010000
1,경기,2016.0,1.0,1.0,NaN,565488.0,6306634.0,8.966558,38.0,79.033906,80.588235,20.108824,-0.091837,10.661702,1031.633333,7.016667
2,경남,2016.0,1.0,1.0,NaN,185840.0,1695187.0,10.962802,39.3,55.186508,69.500000,31.638889,0.815000,13.610000,1031.175000,5.616667
3,경북,2016.0,1.0,1.0,NaN,195135.0,1356568.0,14.384461,41.5,62.858333,68.962963,28.396296,0.371429,13.244643,1030.309524,6.233333
4,광주,2016.0,1.0,1.0,NaN,69876.0,729995.0,9.572120,37.4,57.875000,85.333333,23.400000,1.300000,10.300000,1032.200000,5.850000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6217,전남,2016.0,12.0,31.0,NaN,158747.0,951889.0,16.677050,42.6,34.282500,79.909091,21.753535,2.539189,10.937838,1030.607692,2.573333
6218,전북,2016.0,12.0,31.0,NaN,139233.0,927505.0,15.011563,41.4,44.546347,87.722222,15.777778,1.012903,9.300000,1030.586667,3.466667
6219,제주,2016.0,12.0,31.0,NaN,36378.0,322545.0,11.278426,39.2,24.247294,67.741935,22.135484,3.997143,7.931429,1029.900000,3.046667
6220,충남,2016.0,12.0,31.0,NaN,146606.0,1064765.0,13.768860,40.7,43.536458,82.550000,17.345000,2.663636,7.596970,1030.100000,2.520000


In [88]:
dataset5 = dataset4.drop(columns=['frequency'])

In [91]:
final_model = finalize_model(blender_5)
prediction = predict_model(final_model, data = dataset5)

In [92]:
prediction

,area,yyyy,mm,dd,elderly_pop,total_pop,elderly_ratio,avg_age,pm10,avg_rhm,rhm_min_avg_dif,avg_ta,min_max_Ta,avg_ps,min_max_ps,Label
0,1.104308,2016.0,1.0,1.0,109543.0,781185.0,14.022671,41.400002,54.629547,73.529411,19.665951,-1.454167,10.104167,1030.229980,8.010000,1.397007
1,4.613982,2016.0,1.0,1.0,565488.0,6306634.0,8.966558,38.000000,79.033905,80.588234,20.108824,-0.091837,10.661702,1031.633301,7.016667,4.758306
2,2.000000,2016.0,1.0,1.0,185840.0,1695187.0,10.962802,39.299999,55.186508,69.500000,31.638889,0.815000,13.610000,1031.175049,5.616667,2.298198
3,1.888636,2016.0,1.0,1.0,195135.0,1356568.0,14.384461,41.500000,62.858334,68.962959,28.396296,0.371429,13.244643,1030.309570,6.233333,2.040185
4,0.663366,2016.0,1.0,1.0,69876.0,729995.0,9.572121,37.400002,57.875000,85.333336,23.400000,1.300000,10.300000,1032.199951,5.850000,0.917083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6217,1.429765,2016.0,12.0,31.0,158747.0,951889.0,16.677050,42.599998,34.282501,79.909088,21.753536,2.539189,10.937838,1030.607666,2.573333,1.438799
6218,1.160213,2016.0,12.0,31.0,139233.0,927505.0,15.011563,41.400002,44.546345,87.722221,15.777778,1.012903,9.300000,1030.586670,3.466667,1.202192
6219,0.274436,2016.0,12.0,31.0,36378.0,322545.0,11.278426,39.200001,24.247293,67.741936,22.135485,3.997143,7.931428,1029.900024,3.046667,0.248643
6220,1.115207,2016.0,12.0,31.0,146606.0,1064765.0,13.768860,40.700001,43.536457,82.550003,17.344999,2.663636,7.596970,1030.099976,2.520000,1.207986


In [94]:
save_model(final_model, 'my_best_pipeline')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=/var/folders/f5/x0_185757lld2fhv6ygmb8f40000gn/T/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['yyyy', 'mm', 'dd', 'elderly_pop',
                                              'total_pop', 'elderly_ratio',
                                              'avg_age', 'pm10', 'avg_rhm',
                                              'rhm_min_avg_dif', 'avg_ta',
                                              'min_max_Ta', 'avg_ps',
                                              'min_max_ps'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  Tra...
                                     transformer=VarianceThreshold(threshold=0))),
                 ('actual_estimator',
                  VotingRegressor(estimators=[('Extra Trees Regressor',
                                               ExtraTreesRegressor(n_jobs=-1,
                          

### To load the model back in the environment:

In [95]:
loaded_model = load_model('my_best_pipeline')
print(loaded_model)

Transformation Pipeline and Model Successfully Loaded
Pipeline(memory=Memory(location=/var/folders/f5/x0_185757lld2fhv6ygmb8f40000gn/T/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['yyyy', 'mm', 'dd', 'elderly_pop',
                                             'total_pop', 'elderly_ratio',
                                             'avg_age', 'pm10', 'avg_rhm',
                                             'rhm_min_avg_dif', 'avg_ta',
                                             'min_max_Ta', 'avg_ps',
                                             'min_max_ps'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 Tra...
                                    transformer=VarianceThreshold(threshold=0))),
                ('actual_estimator',
                 VotingRegressor(estimators=[('Extra Trees Regressor',
                                              ExtraTreesRegresso